In [1]:
import requests
import json
# r = requests.get("https://api.opendota.com/api/players/91886888/matches?region=2&limit=5")
# with open('matches_sri_5.json', 'w') as data_file:
#     json.dump(r.json(),data_file)

In [2]:
# import bigjson

# with open('E:\yasp-dump.json\yasp-dump.json', 'rb') as f:
#     j = bigjson.load(f)
#     element = j[4]
#     print(element)

# import ijson

# with open('E:\yasp-dump.json\yasp-dump.json', 'rb') as input_file:
#         # load json iteratively
#         parser = ijson.parse(input_file)
#         chat = ijson.items(input_file, 'features.item.chat.item')
#         for item in chat:
#             print('Chat: {}'.format(item))
# #         for prefix, event, value in parser:
# #             print('prefix={}, event={}, value={}'.format(prefix, event, value))

In [9]:
wins = {}
losses = {}
both = {}

# with open('matches_sri_5.json', 'r') as data_file:
with open('games.json', 'r', encoding="utf-8") as data_file:
    data = json.load(data_file)
    
    for element in data:
#         print(element)
        r = requests.get("https://api.opendota.com/api/matches/" + str(element['match_id']))
        r = r.json()

        r.pop('all_word_counts', None)
        r.pop('match_id', None)
        r.pop('barracks_status_dire', None)
        r.pop('barracks_status_radiant', None)
#         r.pop('chat', None)
        r.pop('cluster', None)
        r.pop('cosmetics', None)
        r.pop('dire_score', None)
        r.pop('dire_team_id', None)
        r.pop('draft_timings', None)
        r.pop('duration', None)
        r.pop('engine', None)
#         r.pop('first_blood_time', None)
#         r.pop('game_mode', None)
        r.pop('human_players', None)
        r.pop('leagueid', None)
        r.pop('lobby_type', None)
        r.pop('match_seq_num', None)
        r.pop('negative_votes', None)
        r.pop('objectives', None)
        r.pop('picks_bans', None)
        r.pop('positive_votes', None)
        r.pop('radiant_gold_adv', None)
        r.pop('radiant_score', None)
        r.pop('radiant_team_id', None)
#         r.pop('radiant_win', None)
        r.pop('radiant_xp_adv', None)
        r.pop('skill', None)
        r.pop('start_time', None)
        r.pop('teamfights', None)
        r.pop('tower_status_dire', None)
        r.pop('tower_status_radiant', None)
        r.pop('version', None)
        r.pop('replay_salt', None)
        r.pop('series_id', None)
        r.pop('series_type', None)
        r.pop('players', None)
        r.pop('patch', None)
        r.pop('region', None)
        r.pop('throw', None)
        r.pop('loss', None)
        r.pop('win', None)
        r.pop('my_word_counts', None)
        r.pop('comeback', None)
        r.pop('stomp', None)
        r.pop('replay_url', None)
     
        pop_chat = []
        for index, item in enumerate(r['chat']):
            if item['slot'] == 10:
                pop_chat.append(index)

        print("pop index" + str(pop_chat))
        for item in reversed(pop_chat):
            print("Popped" + str(r['chat'].pop(int(item))))
        
        with open('chat_wheel.json', 'r',encoding='utf8') as data_file:
            chat_wheel = json.load(data_file)
            
        for idx, item in enumerate(r['chat']):
            if item['type'] == 'chatwheel':
#                 print(item['key'])
                item['type'] = 'chat'
                item['key'] = chat_wheel[item['key']].get('message')
        
        both[element['match_id']] = r
        print("Added to both.")
#         print(r)
        
        if element['radiant_win'] == True:
            wins[element['match_id']] = r
            print("Added to wins.")
        else:
            losses[element['match_id']] = r
            print("Added to losses.")
        with open('wins.json', 'w') as outfile:
            json.dump(wins, outfile)
        with open('losses.json', 'w') as outfile:
            json.dump(losses, outfile)
        with open('both.json', 'w') as outfile:
            json.dump(both, outfile)
            
# print("Wins:")
# print(wins)
# print("Losses:")
# print(losses)
# print("Both:")
# print(both)


<Response [404]>


KeyError: 'chat'

In [ ]:
wins

In [ ]:
import copy
win_chats={}
lose_chats={}
for game in wins:
    game_copy = copy.deepcopy(wins[game])
    game_copy['chat']=[line for line in wins[game]['chat'] if not (line['slot']>4)]
    #print(game_copy)
    win_chats[game]=game_copy
    
    game_copy = copy.deepcopy(wins[game])
    game_copy['chat']=[line for line in wins[game]['chat'] if not (line['slot']<=4)]
    #print(game_copy)
    lose_chats[game]=game_copy

for game in losses:
    game_copy = copy.deepcopy(losses[game])
    game_copy['chat']=[line for line in losses[game]['chat'] if not (line['slot']>4)]
    lose_chats[game]=game_copy
    
    game_copy = copy.deepcopy(losses[game])
    game_copy['chat']=[line for line in losses[game]['chat'] if not (line['slot']<=4)]
    #print(game_copy)
    win_chats[game]=game_copy

print(win_chats) 
print(len(win_chats))
print(lose_chats) 
len(lose_chats)


In [ ]:
wordDict = {}
for game in both.keys():
    for i in both[game]['chat']:
        for word in i['key'].split():
            if word in wordDict:
                wordDict[word] = wordDict[word]+1
            else:
                wordDict[word]=1

In [ ]:
for i in both.keys():
    print(i)

In [ ]:
wordDict

In [ ]:
import operator
wordDictCounts = sorted(wordDict.items(), key=operator.itemgetter(1))

In [ ]:
wordDictCounts[::-1]

In [ ]:
words = list(wordDict.keys())

In [ ]:
words

In [ ]:
words.insert(0,'Radiant Wins')

In [ ]:
import numpy as np
AllWords = np.zeros((len(both)*2,len(wordDict)+1), dtype=int)
game_index = 0
for game in win_chats.keys():
    AllWords[game_index][0]=1
    for line in win_chats[game]['chat']:
        for word in line['key'].split():
            AllWords[game_index][words.index(word)]=AllWords[game_index][words.index(word)]+1
    game_index+=1
for game in lose_chats.keys():
    for line in lose_chats[game]['chat']:
        for word in line['key'].split():
            AllWords[game_index][words.index(word)]=AllWords[game_index][words.index(word)]+1
    game_index+=1

In [ ]:
AllWords

In [ ]:
import pandas as pd 
df = pd.DataFrame(AllWords, columns = words) 

In [ ]:
df

In [ ]:
X = df.iloc[:,1:].values
y = df.iloc[:,0].values

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [ ]:
y_test

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train, y_train)

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))